# Building a car washer model with SimPy

Consider that a company purchased a commercial car washer and wants to optimize its operation to increase profitability. Building a discrete-event model can be helpful because it can help identify bottlenecks, streamline resources, and incrementally adjust processes towards reaching full capacity.

The commercial car washer takes five minutes to complete a car wash cycle.

Build a discrete-event model that mimics the behavior of this machine, and run it for eight hours (480 minutes) to predict the number of cars washed, and log the time of completion of each cycle.

In [1]:
# Import SimPy
import simpy

def car_wash(env):
    car_wash_num = 0
    while True:
      car_wash_num += 1

      # Get the current simulation time and add process time
      print(f'Time {env.now:02d} min | Car Wash # {car_wash_num:02d}')
      yield env.timeout(5)

# Create SimPy Environment and add process generator
env = simpy.Environment()
env.process(car_wash(env))

# Run model
env.run(until=8*60)

Time 00 min | Car Wash # 01
Time 05 min | Car Wash # 02
Time 10 min | Car Wash # 03
Time 15 min | Car Wash # 04
Time 20 min | Car Wash # 05
Time 25 min | Car Wash # 06
Time 30 min | Car Wash # 07
Time 35 min | Car Wash # 08
Time 40 min | Car Wash # 09
Time 45 min | Car Wash # 10
Time 50 min | Car Wash # 11
Time 55 min | Car Wash # 12
Time 60 min | Car Wash # 13
Time 65 min | Car Wash # 14
Time 70 min | Car Wash # 15
Time 75 min | Car Wash # 16
Time 80 min | Car Wash # 17
Time 85 min | Car Wash # 18
Time 90 min | Car Wash # 19
Time 95 min | Car Wash # 20
Time 100 min | Car Wash # 21
Time 105 min | Car Wash # 22
Time 110 min | Car Wash # 23
Time 115 min | Car Wash # 24
Time 120 min | Car Wash # 25
Time 125 min | Car Wash # 26
Time 130 min | Car Wash # 27
Time 135 min | Car Wash # 28
Time 140 min | Car Wash # 29
Time 145 min | Car Wash # 30
Time 150 min | Car Wash # 31
Time 155 min | Car Wash # 32
Time 160 min | Car Wash # 33
Time 165 min | Car Wash # 34
Time 170 min | Car Wash # 35
Time 

SimPy model reveals that the new commercial car washer can complete 96 cycles in eight hours.

# Modeling a car production line: Python generators

You have been asked to build a discrete-event model to help optimize a car production line. To get started, you had to identify the main groups of processes involved in the production line. These are (1) welding and painting and (2) assembly and testing. Of course, each of these groups of processes involves many sub-processes and tasks, but for now, you are focused on coding a first, high-level version of the model.

Since you have already identified the critical groups of processes, it's time to determine the average time each process takes to complete. You did your research and came up with 15 hours for welding and painting and 24 hours for assembling parts and testing.

The simpy package has been imported for you.

Time in the model is in hours.

In [2]:
# Defining a Generator that includes the processes
def  car_production_line_gen(env):
  car_number = 0
  while True:
    car_number += 1

    # Process 1: Clock the time requirement for welding and painting
    yield  env.timeout(15)
    print(f"Car {car_number}: Welding and painting (completed) => time = {env.now}")

    # Process 2: Clock in time for process 2 and yield it
    yield env.timeout(24)
    print(f"Car {car_number}: Assembly of parts and testing (completed) => time = {env.now}")

    # Print car ready for shipment
    print(f"Car {car_number}: Car ready for shipping! time = {env.now} hours")

# Modeling a car production line: Create and run the model

In this exercise, you'll continue to work on the car production line example from the previous exercise.

Now that you have created the generator car_production_line_gen(), which clocks in and yields the duration of processes involved in the production line ("welding and painting" and "assembly and testing"), it's time to run your SimPy model!

In [3]:
# Create your SimPy environment
env = simpy.Environment()

# Add the process to the environment
env.process(car_production_line_gen(env))

# Run your discrete-event model
env.run(until=1000)

Car 1: Welding and painting (completed) => time = 15
Car 1: Assembly of parts and testing (completed) => time = 39
Car 1: Car ready for shipping! time = 39 hours
Car 2: Welding and painting (completed) => time = 54
Car 2: Assembly of parts and testing (completed) => time = 78
Car 2: Car ready for shipping! time = 78 hours
Car 3: Welding and painting (completed) => time = 93
Car 3: Assembly of parts and testing (completed) => time = 117
Car 3: Car ready for shipping! time = 117 hours
Car 4: Welding and painting (completed) => time = 132
Car 4: Assembly of parts and testing (completed) => time = 156
Car 4: Car ready for shipping! time = 156 hours
Car 5: Welding and painting (completed) => time = 171
Car 5: Assembly of parts and testing (completed) => time = 195
Car 5: Car ready for shipping! time = 195 hours
Car 6: Welding and painting (completed) => time = 210
Car 6: Assembly of parts and testing (completed) => time = 234
Car 6: Car ready for shipping! time = 234 hours
Car 7: Welding an

SimPy allows you to run a discrete-event model in three lines of code, confirming that 26 cars can be manufactured in 1000 days! You can apply the same concepts and steps to create a model for any business involving a sequence of discrete events, creating a 'Digital Twin' that can be powerful for optimization.

# Managing payment queues

A clothes shop becomes very busy at peak hours when people often queue to pay. Currently, there is only one cashier, and you have been asked to do a cost-benefit analysis to determine how many cashiers would be needed to reduce waiting times as much as possible to increase profitability.

You have decided to build a discrete-event model. You know that:

On average, a new customer joins the queue every 15 seconds during peak hours;
Customers usually bring several items with them, typically between 1 and 20; and
It takes an average of 3 seconds to scan an item in the cashier, and the payment generally takes another 20 seconds.
The argument counter stores the SimPy resource, and the argument customer_num tracks the number of customers.

Let's run the model and calculate how long it takes to serve 30 customers with a different number of cashiers.

In [4]:
import random

TIME_PAY = 20
TIME_SCAN_PER_ITEM = 3
NEW_CUSTOMERS = 30
INTERVAL_CUSTOMERS = 15

def source(env, total_num_customers, interval, counter):

    """Source generates customers randomly"""
    for i in range(total_num_customers):
        c = customer(env, i+1, counter)
        env.process(c)
        # time between new people arriving and clocking it in
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)

def customer(env, customer_num, counter):

    num_items = random.randint(1.0, 20)
    print(f"Time: {env.now:7.4f} sec | Customer {customer_num:02d} > Joining the queue with {num_items:02d} items!")

    # Open the resource counter request
    with counter.request() as request:

        yield request
        print(f"Time: {env.now:7.4f} sec | Customer {customer_num:02d} > Got to cashier!")
        time_counter = TIME_PAY + random.randint(1.0, 20) * TIME_SCAN_PER_ITEM

        # Yield the processing time
        yield env.timeout(time_counter)
        print(f"Time: {env.now:7.4f} sec | Customer {customer_num:02d} > Finished ")

env = simpy.Environment()

# Create resource
counter = simpy.Resource(env, capacity=1)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter))
env.run()

Time:  0.0000 sec | Customer 01 > Joining the queue with 12 items!
Time:  0.0000 sec | Customer 01 > Got to cashier!
Time:  2.5663 sec | Customer 02 > Joining the queue with 01 items!
Time:  5.6963 sec | Customer 03 > Joining the queue with 01 items!
Time: 28.1127 sec | Customer 04 > Joining the queue with 09 items!
Time: 50.0000 sec | Customer 01 > Finished 
Time: 50.0000 sec | Customer 02 > Got to cashier!
Time: 50.4119 sec | Customer 05 > Joining the queue with 08 items!
Time: 52.3645 sec | Customer 06 > Joining the queue with 08 items!
Time: 53.1827 sec | Customer 07 > Joining the queue with 11 items!
Time: 54.5570 sec | Customer 08 > Joining the queue with 13 items!
Time: 68.5297 sec | Customer 09 > Joining the queue with 05 items!
Time: 78.5650 sec | Customer 10 > Joining the queue with 18 items!
Time: 80.7984 sec | Customer 11 > Joining the queue with 06 items!
Time: 85.3011 sec | Customer 12 > Joining the queue with 01 items!
Time: 88.8852 sec | Customer 13 > Joining the queue 

# Question

What is the minimum number of counters required so that they stop being the limiting factor in the speed of the queue?

To help answer this question, the SimPy code you developed in the previous step is provided. Some object names have been changed.

Note that the model contains randomizing functions, covered in future videos, and we need to use random.seed(RANDOM_SEED) to enable a deterministic answer to this exercise - we've set RANDOM_SEED=42.



In [5]:
random.seed(42)
env_my_model = simpy.Environment()
counter_my_model = simpy.Resource(env_my_model, capacity=3)
env_my_model.process(source(env_my_model, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, counter_my_model))
env_my_model.run()

Time:  0.0000 sec | Customer 01 > Joining the queue with 01 items!
Time:  0.0000 sec | Customer 01 > Got to cashier!
Time: 15.3009 sec | Customer 02 > Joining the queue with 05 items!
Time: 15.3009 sec | Customer 02 > Got to cashier!
Time: 19.5143 sec | Customer 03 > Joining the queue with 18 items!
Time: 19.5143 sec | Customer 03 > Got to cashier!
Time: 36.4519 sec | Customer 04 > Joining the queue with 02 items!
Time: 47.0000 sec | Customer 01 > Finished 
Time: 47.0000 sec | Customer 04 > Got to cashier!
Time: 47.3009 sec | Customer 02 > Finished 
Time: 48.5143 sec | Customer 03 > Finished 
Time: 49.8439 sec | Customer 05 > Joining the queue with 08 items!
Time: 49.8439 sec | Customer 05 > Got to cashier!
Time: 51.3196 sec | Customer 06 > Joining the queue with 18 items!
Time: 51.3196 sec | Customer 06 > Got to cashier!
Time: 65.1399 sec | Customer 07 > Joining the queue with 18 items!
Time: 70.0000 sec | Customer 04 > Finished 
Time: 70.0000 sec | Customer 07 > Got to cashier!
Time:

Exactly, three cashiers are all you need to reduce the waiting times to their minimum. More cashiers would be useless because you'd have additional costs without benefit since cashiers would no longer be the factor limiting your sales.

# Modeling a petrol station: Python generators

Consider that a client wants to build a gas station, and you have been asked to create a discrete-event model to help determine the optimal number of gas pumps and the size of the common fuel tank used by the pumps. This model requires simulating the cars arriving at the gas station and the stations' resources: the pumps and the fuel tank. In this exercise, we will focus on the following two steps:

Step 1: Create a generator to simulate the arrival of the cars at the gas station, requesting a pump, and refilling the car tanks.

Step 2: Create a generator to check the level of the tank and request a refill tank when needed. Also, the behavior of the refill tank needs to be modeled.

In the next exercise, you will create the SimPy environment, add processes and resources, and run simulations.

The number of gas pumps are limited and simulated using a SimPy resource stored in the variable gas_station_pumps.

In [6]:
FUEL_TANK_LEVEL = [5, 25]
#FUEL_TANK_SIZE = 50
#REFUELING_SPEED = 2
FUEL_TANK_SIZE = 500
REFUELING_SPEED = 500
THRESHOLD_PERCENT = 10
TANK_TRUCK_TIME = 100

def car(name, env, gas_station_pumps, gas_station_tank):

    fuel_tank_level = random.randint(*FUEL_TANK_LEVEL)
    print(f"{name} arriving at gas station at {env.now}")

    # Request pump
    with gas_station_pumps.request() as req:
        start_time = env.now

        # Yield the pump request
        yield req

        liters_required = FUEL_TANK_SIZE - fuel_tank_level

        # Remove liters_required from the tank
        yield gas_station_tank.get(liters_required)

        yield env.timeout(liters_required / REFUELING_SPEED)
        print(f"{name} finished refueling in {env.now - start_time} seconds.")

In [7]:


def gas_station_pumps_control(env, gas_station_tank):

  while True:
    # Check if the level of the tank is below the threshold
    if gas_station_tank.level / gas_station_tank.capacity * 100 < THRESHOLD_PERCENT:

        print(f"Calling tank truck at {env.now}")
        yield env.process(tank_truck(env, gas_station_tank))

    yield env.timeout(10)

def tank_truck(env, gas_station_tank):

  yield env.timeout(TANK_TRUCK_TIME)
  print(f"Tank truck arriving at time {env.now}")
  vol_to_refill = gas_station_tank.capacity - gas_station_tank.level
  print(f"Tank truck refuelling {vol_to_refill} liters")

  # Add fuel to the tank
  yield gas_station_tank.put(vol_to_refill)

In [8]:
import itertools
T_INTER = [1, 20]

def car_generator(env, gas_station_pumps, gas_station_tank):
    """Generate new cars that arrive at the gas station."""
    for i in itertools.count():
        yield env.timeout(random.randint(*T_INTER))
        env.process(car('Car %d' % i, env, gas_station_pumps, gas_station_tank))

You have now created the two generators needed to build the SimPy model. One generator simulates the arrival of the cars at the gas station and their pump requests, and the other checks the fuel level in the gas station common tank so that a refill truck can be ordered when needed

# Modeling a petrol station: Run the model and analyze the results

In the previous exercise, you created a generator, car_generator(), to simulate the behavior of the cars arriving at a gas station, and another, gas_station_pumps_control(), to manage the gas station fuel storage.

Now, using these generators, we are ready to create a SimPy model, add resources and processes, and run simulations.

In [9]:
GAS_STATION_TANK_SIZE = 500
SIM_TIME = 3000

env = simpy.Environment()

# Create the gas station resource
gas_station_pumps = simpy.Resource(env, capacity=4)

# Create the gas tank container
gas_station_tank = simpy.Container(env, GAS_STATION_TANK_SIZE, init=GAS_STATION_TANK_SIZE)

# Add processes to the SimPy environment
env.process(gas_station_pumps_control(env, gas_station_tank))
env.process(car_generator(env, gas_station_pumps, gas_station_tank))

env.run(until=SIM_TIME)

Car 0 arriving at gas station at 13
Car 0 finished refueling in 0.9819999999999993 seconds.
Calling tank truck at 20
Car 1 arriving at gas station at 28
Car 2 arriving at gas station at 37
Car 3 arriving at gas station at 45
Car 4 arriving at gas station at 63
Car 5 arriving at gas station at 82
Car 6 arriving at gas station at 101
Car 7 arriving at gas station at 113
Car 8 arriving at gas station at 118
Tank truck arriving at time 120
Tank truck refuelling 491 liters
Car 1 finished refueling in 92.982 seconds.
Calling tank truck at 130
Car 9 arriving at gas station at 134
Car 10 arriving at gas station at 136
Car 11 arriving at gas station at 141
Car 12 arriving at gas station at 147
Car 13 arriving at gas station at 167
Car 14 arriving at gas station at 180
Car 15 arriving at gas station at 200
Car 16 arriving at gas station at 217
Tank truck arriving at time 230
Tank truck refuelling 491 liters
Car 2 finished refueling in 193.956 seconds.
Car 17 arriving at gas station at 235
Car 18

# Question

What is the minimum number of pumps needed to serve the maximum number of cars possible for this station?

Use different numbers of pumps to run your model in the console to find out when when the number of cars served stops increasing - that will be the optimal number of pumps. In other words, you'll need to test different gas_station_pumps resource capacities.

As before, gas_station_pumps_control() and car_generator() contain randomizing functions, which we will cover in future videos. For now, paste and modify the code below to find the correct answer:

In [10]:
def simulate(capacity: int):
    random.seed(42)
    env = simpy.Environment()
    gas_station_pumps = simpy.Resource(env, capacity=capacity)
    gas_station_tank = simpy.Container(env, GAS_STATION_TANK_SIZE, init=GAS_STATION_TANK_SIZE)
    env.process(gas_station_pumps_control(env, gas_station_tank))
    env.process(car_generator(env, gas_station_pumps, gas_station_tank))
    env.run(until=SIM_TIME)

In [11]:
simulate(4)

Car 0 arriving at gas station at 4
Car 0 finished refueling in 0.9740000000000002 seconds.
Car 1 arriving at gas station at 5
Calling tank truck at 10
Car 2 arriving at gas station at 13
Car 3 arriving at gas station at 18
Car 4 arriving at gas station at 36
Car 5 arriving at gas station at 55
Car 6 arriving at gas station at 57
Car 7 arriving at gas station at 60
Car 8 arriving at gas station at 68
Car 9 arriving at gas station at 88
Car 10 arriving at gas station at 106
Tank truck arriving at time 110
Tank truck refuelling 487 liters
Car 1 finished refueling in 105.976 seconds.
Calling tank truck at 120
Car 11 arriving at gas station at 124
Car 12 arriving at gas station at 132
Car 13 arriving at gas station at 151
Car 14 arriving at gas station at 152
Car 15 arriving at gas station at 166
Car 16 arriving at gas station at 175
Car 17 arriving at gas station at 182
Car 18 arriving at gas station at 186
Car 19 arriving at gas station at 199
Car 20 arriving at gas station at 211
Tank tr

In [12]:
simulate(5)


Car 0 arriving at gas station at 4
Car 0 finished refueling in 0.9740000000000002 seconds.
Car 1 arriving at gas station at 5
Calling tank truck at 10
Car 2 arriving at gas station at 13
Car 3 arriving at gas station at 18
Car 4 arriving at gas station at 36
Car 5 arriving at gas station at 55
Car 6 arriving at gas station at 57
Car 7 arriving at gas station at 60
Car 8 arriving at gas station at 68
Car 9 arriving at gas station at 88
Car 10 arriving at gas station at 106
Tank truck arriving at time 110
Tank truck refuelling 487 liters
Car 1 finished refueling in 105.976 seconds.
Calling tank truck at 120
Car 11 arriving at gas station at 124
Car 12 arriving at gas station at 132
Car 13 arriving at gas station at 151
Car 14 arriving at gas station at 152
Car 15 arriving at gas station at 166
Car 16 arriving at gas station at 175
Car 17 arriving at gas station at 182
Car 18 arriving at gas station at 186
Car 19 arriving at gas station at 199
Car 20 arriving at gas station at 211
Tank tr

There is no need for more than five pumping stations because six, seven, eight, or more would serve exactly the same number of people. That's because the pumping stations are no longer the limiting factor in your system. This is a great example of how discrete-event simulations can help manage and optimise business processes. Knowing the number of pumps would help you minimize investment and maximize profit, estimate how much fuel has been sold, and how many customers could be served.

# Restaurant model: Managing tables and waiting times
Imagine you want to open a restaurant in a popular area of San Francisco. Deciding the number of tables and kitchen capacity is extremely important to ensure the maximum number of customers are served while minimizing initial investment and running costs. A discrete-event model can help in this investment decision by simulating the level of table occupancy, customer waiting times, and the number of customers leaving the queue due to excessive waiting times.

Let's first define the generator that simulates table requests and customer decisions to wait or leave based on waiting times. In the next exercise, you will set up the model, run it and analyze the results. The time in the model is in minutes.

In [13]:
import random

MIN_PATIENCE = 1
MAX_PATIENCE = 10
MIN_SEATING_TIME = 30
MAX_SEATING_TIME = 90

def customer(env, name, tables):

  global customers_served, customers_quiting_waiting
  arrive = env.now

  # Request a table
  with tables.request() as req:
    patience = random.uniform(MIN_PATIENCE, MAX_PATIENCE)

    # Wait until a table is free or the customer runs out of patience
    results = yield req | env.timeout(patience)
    wait = env.now - arrive

    if req in results:

      print(f"{env.now:7.4f} {name} > Waited {wait:6.3f} minutes for a table!")
      time_at_tables = random.uniform(MIN_SEATING_TIME, MAX_SEATING_TIME)

      # Yield the time the table is occupied by the customer
      yield env.timeout(time_at_tables)
      print(f"{env.now:7.4f} {name} > Finished meal :)")
      customers_served += 1

    else:
      print(f"{env.now:7.4f} {name} > Gave up waiting and left after waiting {wait:7.4f} minutes :(")
      customers_quiting_waiting += 1

# Restaurant model: Set up, run and analyze results
In the previous exercise, you defined the generator that simulates table requests and customer decisions to wait or leave based on waiting times.

Now let's set up the model, run it, and analyze the results. Recall that the objective of building this model is to determine the appropriate number of tables and kitchen capacity to serve the maximum number of customers while minimizing initial investment and running costs.

To set up your model in a meaningful way, you decided to visit restaurants in the area and observe customer behavior.

You noticed that on average:

- During peak hours, new customers arrived every 10 minutes
- Customers had the patience to wait between 1 to 10 minutes for a table (MIN_PATIENCE and MAX_PATIENCE)
- Customers left if the waiting time was longer than 10 min
- Customers occupied the tables for 40 to 90 minutes (MIN_SEATING_TIME and MAX_SEATING_TIME)

The time in the model is in minutes.

In [14]:
def source(env, number, interval, tables):

    """Source generates customers randomly"""
    for i in range(number):
        c = customer(env, f"Customer {i:02d}", tables)
        env.process(c)
        t = random.expovariate(1.0 / interval)
        yield env.timeout(t)

In [15]:
INTERVAL_CUSTOMERS = 10

# Assign the appropriate values to the model parameters
MIN_PATIENCE = 1
MAX_PATIENCE = 10
MIN_SEATING_TIME = 40
MAX_SEATING_TIME = 90

customers_served = 0
customers_quiting_waiting = 0

env = simpy.Environment()

# Create the SimPy resource
tables = simpy.Resource(env, capacity=2)
env.process(source(env, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, tables))

# Run the model
env.run(until=240)
print(f"Total number of tables served: {customers_served:02d}")
print(f"Total number of customers quiting waiting: {customers_quiting_waiting:02d}")

 0.0000 Customer 00 > Waited  0.000 minutes for a table!
16.8743 Customer 01 > Waited  0.000 minutes for a table!
45.6609 Customer 02 > Gave up waiting and left after waiting  5.0506 minutes :(
61.3370 Customer 00 > Finished meal :)
61.3370 Customer 03 > Waited  0.958 minutes for a table!
64.8322 Customer 01 > Finished meal :)
74.3130 Customer 04 > Waited  0.000 minutes for a table!
96.7711 Customer 05 > Gave up waiting and left after waiting  7.4219 minutes :(
99.1677 Customer 06 > Gave up waiting and left after waiting  4.0226 minutes :(
99.5425 Customer 07 > Gave up waiting and left after waiting  1.7252 minutes :(
115.2026 Customer 03 > Finished meal :)
120.1690 Customer 08 > Waited  0.000 minutes for a table!
126.2753 Customer 04 > Finished meal :)
126.2753 Customer 09 > Waited  4.471 minutes for a table!
129.4047 Customer 10 > Gave up waiting and left after waiting  5.1933 minutes :(
132.7346 Customer 11 > Gave up waiting and left after waiting  4.5053 minutes :(
155.6350 Custome

# Question
Now that your model has been created, let's explore it to answer practical optimization questions.

What is the minimum number of tables needed so that no customers are quitting waiting for a table during a four-hour period?

You will need to run the model with incrementally more tables and check in the console when the number of customers served stops increasing.

The code below resets the variables, creates the SimPy environment, modifies the resource object with a different number of tables, and adds processes before running the model - just like you did before (see code below).



In [16]:
RANDOM_SEED = 42
NEW_CUSTOMERS = 1000

customers_served = 0
customers_quiting_waiting = 0

def simulate_restaurant(capacity: int, time_span=60*4):
    random.seed(RANDOM_SEED)
    env_my_model = simpy.Environment()
    tables_my_model = simpy.Resource(env_my_model, capacity=capacity)
    env_my_model.process(source(env_my_model, NEW_CUSTOMERS, INTERVAL_CUSTOMERS, tables_my_model))
    env_my_model.run(until=time_span)
    print(f"Total number of servings: {customers_served:02d}")
    print(f"Total number of customers quiting waiting: {customers_quiting_waiting:02d}")

In [17]:
customers_served = 0
customers_quiting_waiting = 0

simulate_restaurant(8)

 0.0000 Customer 00 > Waited  0.000 minutes for a table!
10.2006 Customer 01 > Waited  0.000 minutes for a table!
12.7265 Customer 02 > Waited  0.000 minutes for a table!
34.9993 Customer 03 > Waited  0.000 minutes for a table!
35.3018 Customer 04 > Waited  0.000 minutes for a table!
35.5708 Customer 05 > Waited  0.000 minutes for a table!
43.4441 Customer 06 > Waited  0.000 minutes for a table!
53.7515 Customer 00 > Finished meal :)
60.0215 Customer 07 > Waited  0.000 minutes for a table!
71.9994 Customer 08 > Waited  0.000 minutes for a table!
73.8226 Customer 02 > Finished meal :)
84.0356 Customer 01 > Finished meal :)
100.2671 Customer 03 > Finished meal :)
103.5146 Customer 09 > Waited  0.000 minutes for a table!
104.5318 Customer 10 > Waited  0.000 minutes for a table!
105.0341 Customer 05 > Finished meal :)
107.7954 Customer 07 > Finished meal :)
107.7961 Customer 04 > Finished meal :)
116.6367 Customer 08 > Finished meal :)
120.9891 Customer 11 > Waited  0.000 minutes for a tab

Adding more than eight tables is useless because, from that point on, tables are no longer the limiting factor as no customers are quitting the queue anymore.

# Build your model: Create an environment and resources
You have been asked to help optimize the assembly line of an aircraft manufacturer. The main components of the aircraft are the (1) fuselage, (2) wings, (3) empennage (the tail end), (4) power plant (engine and propeller), and (5) landing gear.

Each of these components goes in a different assembly section that has 3, 2, 2, and 3 slots. This means that once a step is completed, it will go to the following step if a slot is available; otherwise, it will have to wait. The assembly sequence must follow the order of Steps 1-4 displayed in the following diagram. The model time is in hours.

![](../images/aircraft_assembly_line.png)

Build a discrete-event model to simulate the assembly line.

In [18]:
PLANE_ORDERS = 30

def order_aircraft(env, total_mum_orders, step_1_fuselage, step_2_wings, step_3_power_plant, step_4_landing_gear):
    yield env.timeout(0)

# Define a dictionary with your processes
processing_time = {
  "step_1_fuselage": 20,
  "step_2_wings": 8,
  "step_3_power_plant": 10,
  "step_4_landing_gear": 8
}

# Create your SimPy Environment with the name env
env = simpy.Environment()

# Create resources for each assembly step
step_1_fuselage = simpy.Resource(env, capacity=3)
step_2_wings = simpy.Resource(env, capacity=2)
step_3_power_plant = simpy.Resource(env, capacity=2)
step_4_landing_gear = simpy.Resource(env, capacity=3)

env.process(order_aircraft(env, PLANE_ORDERS, step_1_fuselage, step_2_wings, step_3_power_plant, step_4_landing_gear))

env.run()

# Build your model: Generate aircraft orders
Now that the SimPy environment and resources have been created, let's link it with a generator that simulates aircraft purchase orders. There are 30 aircraft orders.

The assembly_line() function makes sequential resource requests for the different aircraft component manufacturing sections. The code below shows one such request.

In [19]:
def assembly_line(env, aircraft_id, step_1_fuselage, step_2_wings, step_3_power_plant, step_4_landing_gear):

    processing_time_step_names = list(processing_time.keys())

    print(f"time: {env.now:7.4f} | Aircraft {aircraft_id:02d} | New order")

    # Open the resource step_1_fuselage request
    with step_1_fuselage.request() as slot_request_1:
        request_1_time = env.now
        yield slot_request_1
        print(f"time: {env.now:7.4f} | Aircraft {aircraft_id:02d} | Enters: step_1_fuselage     | Queued for {env.now-request_1_time} hours")
        yield env.timeout(processing_time[processing_time_step_names[0]])

    with step_2_wings.request() as slot_request_2:
        request_2_time = env.now
        yield slot_request_2
        print(f"time: {env.now:7.4f} | Aircraft {aircraft_id:02d} | Enters: step_2_wings        | Queued for {env.now-request_2_time} hours")
        yield env.timeout(processing_time[processing_time_step_names[1]])

    with step_3_power_plant.request() as slot_request_3:
        request_3_time = env.now
        yield slot_request_3
        print(f"time: {env.now:7.4f} | Aircraft {aircraft_id:02d} | Enters: step_3_power_plant  | Queued for {env.now-request_3_time} hours")
        yield env.timeout(processing_time[processing_time_step_names[2]])

    with step_4_landing_gear.request() as slot_request_4:
        request_4_time = env.now
        yield slot_request_4
        print(f"time: {env.now:7.4f} | Aircraft {aircraft_id:02d} | Enters: step_4_landing_gear | Queued for {env.now-request_4_time} hours")
        yield env.timeout(processing_time[processing_time_step_names[3]])
        print(f"time: {env.now:7.4f} | Aircraft {aircraft_id:02d} | Complete")

In [20]:
def order_aircraft(env, total_num_orders, step_1_fuselage, step_2_wings, step_3_power_plant, step_4_landing_gear):

    # Generate the orders with a for-loop
    for request_i in range(total_num_orders):

        assembly_process_request = assembly_line(env, request_i, step_1_fuselage, step_2_wings, step_3_power_plant, step_4_landing_gear)

        # Initiate an assembly line process for each request
        env.process(assembly_process_request)

        # Clock-in the time between requests
        yield env.timeout(0)

# Build your model: Run the model and examine results
You have created the assembly line environment and resources, the generators that produce aircraft orders, and the generator that characterizes the response of the assembly line given the processing times prescribed and limited resources available. The SimPy environment is stored in variable env.

Recall that your assembly sequence is defined in the figure below. The model time is in hours.

![](../images/aircraft_assembly_line.png)

Let's now run the model.

In [21]:
random.seed(42)

# Create your SimPy Environment with the name env
env = simpy.Environment()

# Create a SimPy resource for the fuselage assembly block
step_1_fuselage = simpy.Resource(env, 3)

# Create a SimPy resource for the wings assembly block
step_2_wings = simpy.Resource(env, 2)

# Create a SimPy resource for power plant assembly block
step_3_power_plant = simpy.Resource(env, 2)

# Create a SimPy resource for landing gear assembly block
step_4_landing_gear = simpy.Resource(env, capacity=3)

env.process(order_aircraft(env, PLANE_ORDERS, step_1_fuselage, step_2_wings, step_3_power_plant, step_4_landing_gear))

<Process(order_aircraft) object at 0x111140c70>

# Question
How long does it take to deliver the 30-aircraft order?

The SimPy model environment is stored in object env. Go to the console and run it.

The entire SimPy code needed is shown below.

In [22]:
env.run()

time:  0.0000 | Aircraft 00 | New order
time:  0.0000 | Aircraft 01 | New order
time:  0.0000 | Aircraft 00 | Enters: step_1_fuselage     | Queued for 0 hours
time:  0.0000 | Aircraft 02 | New order
time:  0.0000 | Aircraft 01 | Enters: step_1_fuselage     | Queued for 0 hours
time:  0.0000 | Aircraft 03 | New order
time:  0.0000 | Aircraft 02 | Enters: step_1_fuselage     | Queued for 0 hours
time:  0.0000 | Aircraft 04 | New order
time:  0.0000 | Aircraft 05 | New order
time:  0.0000 | Aircraft 06 | New order
time:  0.0000 | Aircraft 07 | New order
time:  0.0000 | Aircraft 08 | New order
time:  0.0000 | Aircraft 09 | New order
time:  0.0000 | Aircraft 10 | New order
time:  0.0000 | Aircraft 11 | New order
time:  0.0000 | Aircraft 12 | New order
time:  0.0000 | Aircraft 13 | New order
time:  0.0000 | Aircraft 14 | New order
time:  0.0000 | Aircraft 15 | New order
time:  0.0000 | Aircraft 16 | New order
time:  0.0000 | Aircraft 17 | New order
time:  0.0000 | Aircraft 18 | New order
tim

# Question
What is the bottleneck step of your assembly line?

A bottleneck process is one that often causes congestion. Examine the console model outputs and look for the process with the longest queue times.

# Answer
Step 1 Fuselage takes the longest, Queued up to 180 hours

# Question
What would be the minimum number of new step_1_fuselage parallel production lines needed to minimize the queueing time at this process to 60 hours?

You will need to run the model using an incremental number of slots (i.e., production lines) for this bottleneck process, starting from the current situation of 3 parallel production lines. You should aim for a situation where the queueing time of step_1_fuselage is always less or equal to 60 hours.

You can re-run the model in the console

In [23]:
def simulate_production(step_1_fuselage_capacity: int):
    random.seed(42)
    env = simpy.Environment()
    step_1_fuselage = simpy.Resource(env, capacity=step_1_fuselage_capacity)
    step_2_wings = simpy.Resource(env, capacity=2)
    step_3_power_plant = simpy.Resource(env, capacity=2)
    step_4_landing_gear = simpy.Resource(env, capacity=3)
    env.process(order_aircraft(env, PLANE_ORDERS, step_1_fuselage, step_2_wings, step_3_power_plant, step_4_landing_gear))
    env.run()

In [24]:
simulate_production(step_1_fuselage_capacity=8)

time:  0.0000 | Aircraft 00 | New order
time:  0.0000 | Aircraft 01 | New order
time:  0.0000 | Aircraft 00 | Enters: step_1_fuselage     | Queued for 0 hours
time:  0.0000 | Aircraft 02 | New order
time:  0.0000 | Aircraft 01 | Enters: step_1_fuselage     | Queued for 0 hours
time:  0.0000 | Aircraft 03 | New order
time:  0.0000 | Aircraft 02 | Enters: step_1_fuselage     | Queued for 0 hours
time:  0.0000 | Aircraft 04 | New order
time:  0.0000 | Aircraft 03 | Enters: step_1_fuselage     | Queued for 0 hours
time:  0.0000 | Aircraft 05 | New order
time:  0.0000 | Aircraft 04 | Enters: step_1_fuselage     | Queued for 0 hours
time:  0.0000 | Aircraft 06 | New order
time:  0.0000 | Aircraft 05 | Enters: step_1_fuselage     | Queued for 0 hours
time:  0.0000 | Aircraft 07 | New order
time:  0.0000 | Aircraft 06 | Enters: step_1_fuselage     | Queued for 0 hours
time:  0.0000 | Aircraft 08 | New order
time:  0.0000 | Aircraft 07 | Enters: step_1_fuselage     | Queued for 0 hours
time:  0

 Eight step_1_fuselage parallel production line would be needed to limit queuing times at this bottleneck resource to less or equal to 60 hours.